In [ ]:
import re 
from numpy import log
from pickle import load

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
# Phải dùng định nghĩa hàm chứ không được dùng lambda 
# vì sẽ không thể lưu dưới dạng file pickle được
def _zero():
  return 0

def _ngram():
  return defaultdict(_zero)

In [48]:
# Download VN syllables
!wget -O vn_syllables.txt "https://gist.githubusercontent.com/hieuthi/0f5adb7d3f79e7fb67e0e499004bf558/raw/135a4d9716e49a981624474156d6f247b9b46f6a/all-vietnamese-syllables.txt"

--2020-12-16 13:17:20--  https://gist.githubusercontent.com/hieuthi/0f5adb7d3f79e7fb67e0e499004bf558/raw/135a4d9716e49a981624474156d6f247b9b46f6a/all-vietnamese-syllables.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116290 (114K) [text/plain]
Saving to: ‘vn_syllables.txt’

vn_syllables.txt    100%[===================>] 113.56K  --.-KB/s    in 0.02s   

2020-12-16 13:17:21 (5.64 MB/s) - ‘vn_syllables.txt’ saved [116290/116290]



In [49]:
# tính xác suất dùng smoothing
def get_proba(current_word, next_word):
  if current_word not in bigram_letter :
    return 1 / total_word;
  if next_word not in bigram_letter[current_word]:

    return 1 / (sum(bigram_letter[current_word].values())+ vocab_size)
  return (bigram_letter[current_word][next_word] + 1) / (sum(bigram_letter[current_word].values()) + vocab_size)

In [50]:
def remove_vn_accent(word):
    word = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', word)
    word = re.sub('[éèẻẽẹêếềểễệ]', 'e', word)
    word = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', word)
    word = re.sub('[íìỉĩị]', 'i', word)
    word = re.sub('[úùủũụưứừửữự]', 'u', word)
    word = re.sub('[ýỳỷỹỵ]', 'y', word)
    word = re.sub('đ', 'd', word)
    return word

def gen_accents_word(word):
    word_no_accent = remove_vn_accent(word.lower())
    all_accent_word = {word}
    for w in open('vn_syllables.txt').read().splitlines():
        w_no_accent = remove_vn_accent(w.lower())
        if w_no_accent == word_no_accent:
            all_accent_word.add(w)
    return all_accent_word

In [51]:
# Load language model
with open('/content/drive/MyDrive/Colab Notebooks/NLP/CK/VNTQ_bigram_letter_2.plk','rb') as fout:
  bigram_letter=load(fout)

In [52]:
total_word=0
for k in bigram_letter.keys():
  total_word+=sum(bigram_letter[k].values())

total_word

2603892

In [ ]:
vocab_size = len(bigram_letter)
vocab_size

8817

In [ ]:
# hàm beam search
def beam_search(words, k=3):
  sequences = []
  for idx, word in enumerate(words):
    if idx == 0:
      sequences = [([x], 0.0) for x in gen_accents_word(word)]
    else:
      all_sequences = []
      for seq in sequences:
        for next_word in gen_accents_word(word):
          current_word = seq[0][-1]
          proba = get_proba(current_word, next_word)
          # print(current_word, next_word, proba, log(proba))
          proba = log(proba)
          new_seq = seq[0].copy()
          new_seq.append(next_word)
          all_sequences.append((new_seq, seq[1] + proba))
      # sắp xếp và lấy k kết quả ngon nhất
      all_sequences = sorted(all_sequences,key=lambda x: x[1], reverse=True)
      sequences = all_sequences[:k]
  return sequences

In [53]:
beam_search('khanh map'.split())[0][0]

['khảnh', 'mấp']

In [ ]:
!pip install underthesea

In [ ]:
!python 1712531.py --corpus VNTQ.txt --level char --ngram 3 --filemodel bigram_char.plk

2020-12-16 12:38:32.355943: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
86088it [00:00, 111089.55it/s]
defaultdict(<function _ngram at 0x7f45cae87d90>, {('t', 'ì'): defaultdict(<function _zero at 0x7f45cae87d08>, {'m': 1985, 'n': 3555, 'v': 4, 'k': 3, 't': 3, 'r': 2, 'x': 5, 'l': 2, 'b': 3, 'đ': 4, 'u': 1, 'í': 1}), ('ì', 'm'): defaultdict(<function _zero at 0x7f45cae87d08>, {'t': 360, 'à': 398, 'l': 92, 'ỗ': 16, 'r': 146, 'h': 135, 'ỹ': 8, 'k': 190, 'ộ': 226, 'đ': 423, 'n': 185, 'ọ': 46, 'c': 309, 'm': 128, 'ớ': 165, 'v': 89, 'o': 5, 'a': 72, 'ụ': 30, 'ì': 74, 'ặ': 41, 'u': 111, 'ố': 32, 'b': 58, 'ắ': 14, 'q': 11, 'd': 19, 'ấ': 69, 'ô': 19, 's': 30, 'g': 40, 'i': 24, 'ẹ': 14, 'ư': 23, 'ã': 11, 'x': 30, 'ạ': 18, 'ỉ': 11, 'ú': 3, 'p': 16, 'ð': 13, 'ồ': 4, 'ẫ': 3, 'ở': 8, 'ó': 2, 'ậ': 4, 'ờ': 5, 'ệ': 5, 'y': 11, 'ơ': 1, 'â': 2, 'ừ': 15, 'ù': 6, 'ũ': 7, 'á': 4, 'ê': 2, 'ả': 2, 'ĩ': 1, 'é': 1, 'ề': 1, 'ẻ': 1}), ('m',

In [ ]:
with open('bigram_char.plk',"rb") as fout:
  bigram_char=load(fout)

In [ ]:
bigram_char

defaultdict(<function __main__._ngram>,
            {('t', 'ì'): defaultdict(<function __main__._zero>,
                         {'b': 3,
                          'k': 3,
                          'l': 2,
                          'm': 1985,
                          'n': 3555,
                          'r': 2,
                          't': 3,
                          'u': 1,
                          'v': 4,
                          'x': 5,
                          'í': 1,
                          'đ': 4}),
             ('ì', 'm'): defaultdict(<function __main__._zero>,
                         {'a': 72,
                          'b': 58,
                          'c': 309,
                          'd': 19,
                          'g': 40,
                          'h': 135,
                          'i': 24,
                          'k': 190,
                          'l': 92,
                          'm': 128,
                          'n': 185,
                          